# TP 1: LDA/QDA y optimización matemática de modelos

# Librerias

In [1]:
from base.qda import QDA, TensorizedQDA, FastQDA, EfficientQDA
from base.cholesky import QDA_Chol1, QDA_Chol2, QDA_Chol3, TensorizedChol, EfficientChol
from utils.bench import Benchmark
from utils.datasets import (get_iris_dataset, get_letters_dataset, 
                            get_penguins_dataset, get_wine_dataset,
                            label_encode)

# Consigna QDA

**Notación**: en general notamos

* $k$ la cantidad de clases
* $n$ la cantidad de observaciones
* $p$ la cantidad de features/variables/predictores

**Sugerencia:** combinaciones adecuadas de `transpose`, `stack`, `reshape` y, ocasionalmente, `flatten` y `diagonal` suele ser más que suficiente. Se recomienda *fuertemente* explorar la dimensionalidad de cada elemento antes de implementar las clases.

## Tensorización

En esta sección nos vamos a ocupar de hacer que el modelo sea más rápido para generar predicciones, observando que incurre en un doble `for` dado que predice en forma individual un escalar para cada observación, para cada clase. Paralelizar ambos vía tensorización suena como una gran vía de mejora de tiempos.

### 1) Diferencias entre `QDA`y `TensorizedQDA`

1. ¿Sobre qué paraleliza `TensorizedQDA`? ¿Sobre las $k$ clases, las $n$ observaciones a predecir, o ambas?

   Paraleliza sobre las $k$ clases.

   El método _fit_params(X,y) de la clase derivada `TensorizedQDA` obtiene de la clase base `QDA` las inversas de las matrices de covarianzas y vectores de medias de cada clase como dos listas, cada elemento correspondiente a una clase.
   - Tipo `means` e `inv_covs`: <class 'list'>.
   - Cantidad de elementos de las listas: $k$ (uno por clase).
   - Formato elementos lista `means`: (p, 1).
   - Formato elementos lista `inv_covs`: (p, p).

   Luego, utiliza el método `stack` de numpy para "apilar" los elementos de las listas en un tensor de una dimensión adicional a la de los elementos. El primer eje del tensor corresponde al índice de la clase aplilada (`batch`).
   - Tipo `tensor_means` y `tensor_inv_covs`: <class 'numpy.ndarray'>.
   - Formato `tensor_means`: (k, p, 1).
   - Formato `tensor_inv_covs`: (k, p, p).

   Este arreglo hace que la multiplicación matricial (@) interprete las dos últimas dimensiones del tensor como matrices a multiplicar y las restantes como índices de lote (batch), realizando las operaciones en paralelo sobre las clases. De modo análogo, np.linalg.det calcula los determinantes en paralelo reconociendo los lotes de matrices.


2. Analizar los shapes de `tensor_inv_covs` y `tensor_means` y explicar paso a paso cómo es que `TensorizedQDA` llega a predecir lo mismo que `QDA`.

   Los shapes de `tensor_inv_covs` y `tensor_means` y como son obtenidos se analizaron en el punto anterior.

   `TensorizedQDA` optimiza paralelizando el cálculo de la log-condicional para cada clase dada una observación. Para eso sobreescribe el método `_predict_one` de la clase base, eliminando el lazo `for` donde se realizaba el cálculo de esta probabilidad para cada clase y definiendo el método `_predict_log_conditionals` donde este cálculo se realiza en paralelo a partir de los tensores `tensor_inv_covs` y `tensor_means` mencionados.

   Paso a paso:

   1) Entrenamiento

   Se obtienen las listas de medias e inversas de matrices de covarianzas de cada clase de idéntico modo que en `QDA`. Se sobreescribe el método `_fit_params` para llamar primero al de la clase base QDA y luego generar las versiones tensorizadas.

   ```python
   def _fit_params(self, X, y):
        super()._fit_params(X,y)
        self.tensor_inv_cov = np.stack(self.inv_covs)
        self.tensor_means = np.stack(self.means)
   ```

   Basicamente, el entrenamiento en sí es idéntico en ambos casos.

   2) Predicción:

   Se sobreescribe el método `_predict_one` de la clase `BaseBayesianClassifier`eliminando el lazo `for` y devolviendo el indice de clase como el mayor de la suma del array de probabilidades a priori y del array de log-condicionales de cada clase obtenidas de forma vectorizada en el nuevo método `_predict_log_conditionals`.

   ```python
   def _predict_one(self, x):
        return np.argmax(self.log_a_priori + self._predict_log_conditionals(x)) 
   ```
   En el método `_predict_log_conditionals` obtiene 'unbiased_x' e 'inner_prod' para todas las clases (broadcast, matmul batcheado) de forma simultanea, y luego calcula todas las log-prob de todas las clases juntas

   ```python
   def _predict_log_conditionals(self,x):
        unbiased_x = x - self.tensor_means
        inner_prod = unbiased_x.transpose(0,2,1) @ self.tensor_inv_cov @ unbiased_x
        return 0.5*np.log(LA.det(self.tensor_inv_cov)) - 0.5 * inner_prod.flatten()
   ```

### 2) Optimización

Debido a la forma cuadrática de QDA, no se puede predecir para $n$ observaciones en una sola pasada (utilizar $X \in \mathbb{R}^{p \times n}$ en vez de $x \in \mathbb{R}^p$) sin pasar por una matriz de $n \times n$ en donde se computan todas las interacciones entre observaciones. Se puede acceder al resultado recuperando sólo la diagonal de dicha matriz, pero resulta ineficiente en tiempo y (especialmente) en memoria. Aún así, es *posible* que el modelo funcione más rápido.

3. Implementar el modelo `FasterQDA` (se recomienda heredarlo de `TensorizedQDA`) de manera de eliminar el ciclo for en el método predict.

   Implementado en 'base/qda.py'.



4. Mostrar dónde aparece la mencionada matriz de $n \times n$, donde $n$ es la cantidad de observaciones a predecir.
   
   En el método que calcula la log-condicional para todas las clases y observaciones.

   ```python
   def _predict_log_conditionals_batch(self, x):
        """
        Calcula en paralelo (sobre k y n) los log-condicionales devolviendo una matriz (k, n).

        Notas:
        1) Se desplazan todas las observaciones por la media de cada clase:
               U[k] = X - μ_k     → U.shape = (k, p, n)
        2) Se construye la matriz n×n por clase:
               M[k] = U[k]^T Σ_k^{-1} U[k]   → M.shape = (k, n, n)
        3) Se extrae la diagonal por clase (término cuadrático por observación):
               quad[k, i] = (x_i-μ_k)^T Σ_k^{-1} (x_i-μ_k)  → quad.shape = (k, n)
        4) Se suma el término 0.5·log|Σ_k^{-1}| (expandido a (k,1)) y se resta
           0.5·quad para obtener (k, n).
        """
        # U[k] = X - μ_k  → (k, p, n)  (broadcasting en eje de clases)
        U = x - self.tensor_means

        # M = U^T Σ_k^{-1} U → (k, n, n)  (matmul batcheado sobre k)
        M = U.transpose(0, 2, 1) @ self.tensor_inv_cov @ U

        # Diagonal por clase: q_{k,i} = (x_i-μ_k)^T Σ_k^{-1} (x_i-μ_k) → (k, n)
        quad = np.diagonal(M, axis1=1, axis2=2)

        # 0.5·log|Σ_k^{-1}| → (k,)  ⇒ (k,1) para broadcast sobre n
        logdet = 0.5 * np.log(LA.det(self.tensor_inv_cov)).reshape(-1, 1)

        # Log-condicional
        return logdet - 0.5 * quad  # (k, n)
    '''


5. Demostrar que
   $$
   diag(A \cdot B) = \sum_{cols} A \odot B^T = np.sum(A \odot B^T, axis=1)
   $$ es decir, que se puede "esquivar" la matriz de $n \times n$ usando matrices de $n \times p$. También se puede usar, de forma equivalente,
   $$
   np.sum(A^T \odot B, axis=0).T
   $$
   queda a preferencia del alumno cuál usar.

   El producto matricial es  
   $$
   C = AB, \quad C \in \mathbb{R}^{n \times n}.
   $$  

   La diagonal de $C$ es el vector  
   $$
   diag(C) =
   \begin{bmatrix}
   \sum_{k=1}^p A_{1k} B_{k1} \\
   \sum_{k=1}^p A_{2k} B_{k2} \\
   \vdots \\
   \sum_{k=1}^p A_{nk} B_{kn}
   \end{bmatrix}.
   $$  

   Para cada $i$:  
   $$
   C_{ii} = \sum_{k=1}^p A_{ik} \cdot (B^T)_{ik}.
   $$  

   Esto corresponde al producto escalar entre la fila $i$ de $A$ y la fila $i$ de $B^T$.  
   Por lo tanto,  
   $$
   diag(AB) = \sum_{k=1}^p A_{:,k} \odot B^T_{:,k}.
   $$  

   En notación de NumPy:  
   $$
   diag(AB) = np.sum(A \odot B^T, axis=1).
   $$  

   Aquí $A \odot B^T$ es de dimensión $n \times p$, y al sumar sobre las columnas se obtiene un vector de longitud $n$.  



6. Utilizar la propiedad antes demostrada para reimplementar la predicción del modelo `FasterQDA` de forma eficiente en un nuevo modelo `EfficientQDA`.

   Implementado en base/qda.py

7. Comparar la performance de las 4 variantes de QDA implementadas hasta ahora (no Cholesky) ¿Qué se observa? A modo de opinión ¿Se condice con lo esperado?

   Se observan tiempo y uso de memoria similar en entrenamiento. En prueba las implementaciones tensorizadas bajan el tiempo a costo de aumentar el consumo de memoria. 

In [2]:
# Levantamos el dataset Wine, que tiene 13 features y 178 observaciones en total

X_full, y_full = get_penguins_dataset()

# Encodeamos a número las clases

y_full_encoded = label_encode(y_full)

# Generamos el benchmark

b = Benchmark(
    X_full, y_full_encoded,
    n_runs = 1000,
    warmup = 20,
    mem_runs = 200,
    test_sz = 0.3,
    same_splits = False
)

# Bencheamos implementaciones

to_bench = [QDA, TensorizedQDA, FastQDA, EfficientQDA]

for model in to_bench:
    b.bench(model)

# Hacemos un summary

b.summary()

# Son muchos datos! nos quedamos con un par nomás

summ = b.summary()

# Como es un pandas DataFrame, subseteamos columnas fácil

summ[['train_median_ms', 'test_median_ms', 'train_mem_median_mb', 'test_mem_median_mb', 'mean_accuracy']]

Benching params:
Total runs: 1220
Warmup runs: 20
Peak Memory usage runs: 200
Running time runs: 1000
Train size rows (approx): 240
Test size rows (approx): 102
Test size fraction: 0.3


QDA (MEM):   0%|          | 0/200 [00:00<?, ?it/s]

QDA (TIME):   0%|          | 0/1000 [00:00<?, ?it/s]

TensorizedQDA (MEM):   0%|          | 0/200 [00:00<?, ?it/s]

TensorizedQDA (TIME):   0%|          | 0/1000 [00:00<?, ?it/s]

FastQDA (MEM):   0%|          | 0/200 [00:00<?, ?it/s]

FastQDA (TIME):   0%|          | 0/1000 [00:00<?, ?it/s]

EfficientQDA (MEM):   0%|          | 0/200 [00:00<?, ?it/s]

EfficientQDA (TIME):   0%|          | 0/1000 [00:00<?, ?it/s]

,train_median_ms,test_median_ms,train_mem_median_mb,test_mem_median_mb,mean_accuracy
model,,,,,
QDA,0.49545,7.29685,0.011452,0.003777,0.987049
TensorizedQDA,0.42880,2.44300,0.011360,0.004147,0.986369
FastQDA,0.40715,0.09810,0.011269,0.264603,0.987165
EfficientQDA,0.41525,0.10115,0.011177,0.264603,0.986893


## Cholesky

Hasta ahora todos los esfuerzos fueron enfocados en realizar una predicción más rápida. Los tiempos de entrenamiento (teóricos al menos) siguen siendo los mismos o hasta (minúsculamente) peores, dado que todas las mejoras siguen llamando al método `_fit_params` original de `QDA`.

La descomposición/factorización de [Cholesky](https://en.wikipedia.org/wiki/Cholesky_decomposition#Statement) permite factorizar una matriz definida positiva $A = LL^T$ donde $L$ es una matriz triangular inferior. En particular, si bien se asume que $p \ll n$, invertir la matriz de covarianzas $\Sigma$ para cada clase impone un cuello de botella que podría alivianarse. Teniendo en cuenta que las matrices de covarianza son simétricas y salvo degeneración, definidas positivas, Cholesky como mínimo debería permitir invertir la matriz más rápido.

*Nota: observar que calcular* $A^{-1}b$ *equivale a resolver el sistema* $Ax=b$.

### 3) Diferencias entre implementaciones de `QDA_Chol`

8. Si una matriz $A$ tiene fact. de Cholesky $A=LL^T$, expresar $A^{-1}$ en términos de $L$. ¿Cómo podría esto ser útil en la forma cuadrática de QDA?

   - Si $A$ es simétrica definida positiva y tiene factorización de Cholesky  

     $A=LL^T, L$ triangular inferior,

     entonces su inversa se puede expresar directamente en términos de $L$:  

     $A^{-1} = (L L^{\top})^{-1} = (L^{\top})^{-1} L^{-1} = L^{-\top} L^{-1}$

   - En QDA permite resolver la forma cuadrática sin tener que invertir la matriz de covarianzas. Es mas sencillo invertir y calcular el determinante de una matriz triangular.
     
     $Q = (x-\hat{\mu}_j)^\top \hat{\Sigma}_j^{-1}(x-\hat{\mu}_j) = (x-\hat{\mu}_j)^\top \hat{L}_j^{-T}\hat{L}_j^{-1}(x-\hat{\mu}_j)$

     Sea $u_j = (x-\hat{\mu}_j)$
     
     $Q = u_j^{T}\hat{L}_j^{-T}\hat{L}_j^{-1}u_j = (\hat{L}_j^{-1}u_j)^{T}(\hat{L}_j^{-1}u_j) = \|\hat{L}_j^{-1}u_j\|_2^2.$


7. Explicar las diferencias entre `QDA_Chol1`y `QDA` y cómo `QDA_Chol1` llega, paso a paso, hasta las predicciones.

   - `QDA_Chol1` al momento de entrenar calcula la inversa de las matrices triangualares $L$ en lugar de invertir las matrices de covarianzas completas.
   
    ```python
      self.L_invs = [
            LA.inv(cholesky(np.cov(X[:,y.flatten()==idx], bias=True), lower=True))
                for idx in range(len(self.log_a_priori))
        ]  
    ```

    - `QDA_Chol1` al momento de predecir, en el cálculo de la log-condicional de una clase, obtiene $y = \hat{L}^{-1}(x - \hat{\mu})$. 
      
      Y luego su valor como: $\log |\hat{L}^{-1}| - \frac{1}{2} \|y\|_2^2$

      El cálculo del determinante de la matriz triangular se obtiene como el producto de los elementos de su diagonal. El factor $-\frac{1}{2}$ desaparece al estar calculando el determinante de la inversa y que el $|L|^2 = |\Sigma|$

    ```python
      def _predict_log_conditional(self, x, class_idx):
        L_inv = self.L_invs[class_idx]
        unbiased_x =  x - self.means[class_idx]

        y = L_inv @ unbiased_x

        return np.log(L_inv.diagonal().prod()) -0.5 * (y**2).sum() 
    ```    

8. ¿Cuáles son las diferencias entre `QDA_Chol1`, `QDA_Chol2` y `QDA_Chol3`?

   `QDA_Chol1`: Durante el entrenamiento calcula y guarda la inversa de la matriz triangular inferior de Cholesky de la matriz de covarianzas de cada clase. En la prediccion calcula $y$ para cada clase como el producto de la inversa almacenada `L_inv` y `unbiased_x`.

   `QDA_Chol2`: No invierte la matriz triangular en el entrenamiento y en la predicción obtiene $y$ resolviendo el sistema linal de la matriz triangular para `L` y `unbiased_x`.

   `QDA_Chol3`: Identico a `QDA_Chol1` con la sola diferencia de que utiliza la función $dtrtri$ para calcular especificamente la inversa de una matriz triangular.

9. Comparar la performance de las 7 variantes de QDA implementadas hasta ahora ¿Qué se observa?¿Hay alguna de las implementaciones de `QDA_Chol` que sea claramente mejor que las demás?¿Alguna que sea peor?

Tanto `Chol1`como `Chol3`bajan el tiempo de prueba respecto a `QDA`. `Chol2` tiene peor desenpeño que los otros dos, probablemente porque la cantidad de caracteristicas del dataset es baja y resulta menos costoso invertir la matriz triangular en el entrenamiento y realizar una multiplicacion en la predicción que resolver la matriz triangular en la predicción de `Chol2`.

In [3]:
# Como es una clase, podemos seguir bencheando más después

b.bench(QDA_Chol1)
b.bench(QDA_Chol2)
b.bench(QDA_Chol3)

# Hacemos un summary

summ = b.summary()

# Como es un pandas DataFrame, subseteamos columnas fácil

summ[['train_median_ms', 'test_median_ms', 'train_mem_median_mb', 'test_mem_median_mb', 'mean_accuracy']]

QDA_Chol1 (MEM):   0%|          | 0/200 [00:00<?, ?it/s]

QDA_Chol1 (TIME):   0%|          | 0/1000 [00:00<?, ?it/s]

QDA_Chol2 (MEM):   0%|          | 0/200 [00:00<?, ?it/s]

QDA_Chol2 (TIME):   0%|          | 0/1000 [00:00<?, ?it/s]

QDA_Chol3 (MEM):   0%|          | 0/200 [00:00<?, ?it/s]

QDA_Chol3 (TIME):   0%|          | 0/1000 [00:00<?, ?it/s]

,train_median_ms,test_median_ms,train_mem_median_mb,test_mem_median_mb,mean_accuracy
model,,,,,
QDA,0.49545,7.29685,0.011452,0.003777,0.987049
TensorizedQDA,0.42880,2.44300,0.011360,0.004147,0.986369
FastQDA,0.40715,0.09810,0.011269,0.264603,0.987165
EfficientQDA,0.41525,0.10115,0.011177,0.264603,0.986893
QDA_Chol1,0.70525,5.50220,0.011360,0.003727,0.986612
QDA_Chol2,0.50340,10.19520,0.011543,0.004162,0.986903
QDA_Chol3,0.52190,4.77885,0.011452,0.003674,0.987078


### 4) Optimización

12. Implementar el modelo `TensorizedChol` paralelizando sobre clases/observaciones según corresponda. Se recomienda heredarlo de alguna de las implementaciones de `QDA_Chol`, aunque la elección de cuál de ellas queda a cargo del alumno según lo observado en los benchmarks de puntos anteriores.

    Implementado en base/cholesky.py

13. Implementar el modelo `EfficientChol` combinando los insights de `EfficientQDA` y `TensorizedChol`. Si se desea, se puede implementar `FasterChol` como ayuda, pero no se contempla para el punto.

    Implementado en base/cholesky.py

13. Comparar la performance de las 9 variantes de QDA implementadas ¿Qué se observa? A modo de opinión ¿Se condice con lo esperado?

    Todas las implementaciones tienen precisión similar. En entrenamiento todas tienen tiempos y uso de memoria similares. 

    `TensorizedQDA` y `TensorizedChol` logran la mejorar el tiempo en test respecto `QDA` y las versiones `QDA_Chol` por vectorización.

    Las implementaciones `Fast/Efficient` mejoran aun mas la velocidad, a costo de aumentar el uso de memoria.

    `EfficientChol` pareciera ser la de mejor velocidad con un incremento de memoria menor a `Fast/EfficientQDA`.

In [4]:
# Como es una clase, podemos seguir bencheando más después

b.bench(TensorizedChol)
b.bench(EfficientChol)

# Hacemos un summary

summ = b.summary()

# Como es un pandas DataFrame, subseteamos columnas fácil

summ[['train_median_ms', 'test_median_ms', 'train_mem_median_mb', 'test_mem_median_mb', 'mean_accuracy']]

TensorizedChol (MEM):   0%|          | 0/200 [00:00<?, ?it/s]

TensorizedChol (TIME):   0%|          | 0/1000 [00:00<?, ?it/s]

EfficientChol (MEM):   0%|          | 0/200 [00:00<?, ?it/s]

EfficientChol (TIME):   0%|          | 0/1000 [00:00<?, ?it/s]

,train_median_ms,test_median_ms,train_mem_median_mb,test_mem_median_mb,mean_accuracy
model,,,,,
QDA,0.49545,7.29685,0.011452,0.003777,0.987049
TensorizedQDA,0.42880,2.44300,0.011360,0.004147,0.986369
FastQDA,0.40715,0.09810,0.011269,0.264603,0.987165
EfficientQDA,0.41525,0.10115,0.011177,0.264603,0.986893
QDA_Chol1,0.70525,5.50220,0.011360,0.003727,0.986612
QDA_Chol2,0.50340,10.19520,0.011543,0.004162,0.986903
QDA_Chol3,0.52190,4.77885,0.011452,0.003674,0.987078
TensorizedChol,0.55125,3.54695,0.011374,0.003361,0.986825
EfficientChol,0.38600,0.04950,0.011360,0.042450,0.987000


## Importante:

Las métricas que se observan al realizar benchmarking son muy dependientes del código que se ejecuta, y por tanto de las versiones de las librerías utilizadas. Una forma de unificar esto es utilizando un gestor de versiones y paquetes como _uv_ o _Poetry_, otra es simplemente usando una misma VM como la que provee Colab.

**Cada equipo debe informar las versiones de Python, NumPy y SciPy con que fueron obtenidos los resultados. En caso de que sean múltiples, agregar todos los casos**. La siguiente celda provee una ayuda para hacerlo desde un notebook, aunque como es una secuencia de comandos también sirve para consola.

In [5]:
# Version que deberia funcionar en todos los entornos

import sys
print("Python:", sys.version)

# importlib.metadata es estándar desde Python 3.8; en Colab también está.
try:
    from importlib.metadata import version, PackageNotFoundError
except Exception:
    from importlib_metadata import version, PackageNotFoundError  # backport, por si acaso

for pkg in ["numpy", "scipy"]:
    try:
        print(f"{pkg}:", version(pkg))
    except PackageNotFoundError:
        print(f"{pkg}: NO INSTALADO")

Python: 3.12.7 | packaged by conda-forge | (main, Oct  4 2024, 15:47:54) [MSC v.1941 64 bit (AMD64)]
numpy: 1.26.4
scipy: 1.15.2
